# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
import PyPDF2 as ppdf
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','LesionSizeLocation.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

plotting the predefined image

In [ ]:
def plot_animal_image(ax, animal):
    PATHS=('/NAS02/Rat302/Histology/DS_45.jpg',)
    
    try:
        filePath=[path for path in PATHS if animal in path][0]
    except IndexError:
        logging.error(f'Bad Animal name ({animal}), path not defined!')
        return
    
    f=PIL.Image.open(filePath)
    f.thumbnail((500,500),PIL.Image.ANTIALIAS)
    ax.imshow(f)

---

plot the Lesion Size-Location plot

In [ ]:
def plot_lesion_size_loc(root,ax,AnimalProfile, animalList=None,**kwargs):
    
    if animalList is None:
        animalList=batch_get_animal_list(root,AnimalProfile)
        
    size=[]
    loc=[]
    animals=[]
    color=[]
    for animal in animalList:
        try:
            i=HistologyExcel(NAS_PATH,animal)
        except AssertionError as e:
            logging.warning(repr(e))
            continue
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        
        try:
            t2=i.lesion_location()
            t3=i.lesion_size()
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
            
        loc.append(t2)
        size.append(t3)
        animals.append(animal)
        _,tag=lesion_type(root,animal)
        color.append(ColorCode[tag] if tag in ColorCode else 'gray')

    
    #Plotting
    
    sc=ax.scatter(loc,size,c=color,s=8,**kwargs)
    ax.axvline(0,linestyle='--',color='k',lw=.5);
    
    ax.set_xlim([-.16,.16])
    ax.set_xticks([-.1,-.05,0,.05,.1])
    ax.set_xticklabels(['DMS','$\longleftarrow$','Mid','$\longrightarrow$','DLS'])
    ax.set_xlabel('Lesion centroid')
    ax.tick_params('x',bottom=False)
    ax.spines['bottom'].set_bounds(-.15,.15)
    
    ax.set_ylabel('Lesion size')
    ax.set_ylim([0,1])
    ax.set_yticks(np.arange(0,1.01,.1))
    ax.set_yticklabels([0,'','','','',0.5,'','','','',1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    
    
#     for i, txt in enumerate(animals):
#         ax.annotate(txt[-3:], (loc[i], size[i]))
    
    return loc, size, animals


def add_legend_for_lesion_types(ax):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markerfacecolor='xkcd:gray', marker='o',markeredgewidth=.1,
                                       markeredgecolor='xkcd:black', markersize=3, label='Late')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:black', marker='D',markeredgewidth=.1,
                                       markerfacecolor='xkcd:gray', markersize=3, label='Early')

    leg=ax.legend(handles=[r_marker,b_marker],loc=2,mode=None,ncol=1,
                  facecolor=None,edgecolor=None, fontsize=5,frameon=False,handletextpad=0,columnspacing=1)
    return leg

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    fig=plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111)
    
    profile={'Type':'Good',
             'option': ['not used', 'AsymmetricLesion'],
             'rewardType':'Progressive',
             'initialSpeed':['10'],
             'Speed':'10',
             'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS',
                    'Early-Lesion_DS-NoHab','Early-Lesion_DMS-NoHab']}
    
    loc, size, animals=plot_lesion_size_loc(root,ax,profile)

---

plotting the predefined image

In [ ]:
def plot_example_image(ax):
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

#     path=os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Rat276_25.png')
    
#     f=PIL.Image.open(path)
#     f.thumbnail((500,500),PIL.Image.ANTIALIAS)
#     ax.imshow(f)

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    fig=plt.figure(figsize=(1,1))
    ax=fig.add_subplot(111)
    
    plot_example_image(ax)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:orange'
    DMSColor='purple'
    DSColor='xkcd:green'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    BadLateRats=('Rat223','Rat231')
    NAS_PATH='/NAS02'
       

    
    #===============================================
    
    # GRID 1 PARAMS
       
    profileDLS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    profileLesions1={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS1['Tag'], *profileDMS1['Tag'], *profileDS1['Tag'])))
                     }
    
    
    #================================================
    
    # GRID 2 PARAMS

    profileLesions2={'Type':'Good',
                     'option': ['not used', 'AsymmetricLesion'],
                     'rewardType':'Progressive',
                     'initialSpeed':['10'],
                     'Speed':'10',
                     'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS',
                            'Early-Lesion_DS-NoHab','Early-Lesion_DMS-NoHab']}


Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(4,3.5)
    fig=plt.figure(figsize=figsize,dpi=1200)
    
        

    ##########################################
    # 1: Lesion Size Loc LATE
    gs0= fig.add_gridspec(nrows=1, ncols=2, left=0, bottom=.55, right=1, top=1)
    ax0_= fig.add_subplot(gs0[0])
    ax0= fig.add_subplot(gs0[1])
    
    plot_example_image(ax0_)
    plot_example_image(ax0)
    
    
    
    ##########################################
    # 1: Lesion Size Loc LATE
    gs1= fig.add_gridspec(nrows=1, ncols=1, left=0.025, bottom=0, right=.475, top=.4)
    ax1= fig.add_subplot(gs1[0])
    
    *_,animals1=plot_lesion_size_loc(root,ax1,profileLesions1,marker='o',edgecolors='none',linewidths=.1,alpha=.8)
    
    ax1.set_title(f'Early lesion ($n={len(animals1)}$)', fontsize='xx-small')

    
    
    
    ##########################################
    # 2: Lesion Size Loc EARLY
    gs2= fig.add_gridspec(nrows=1, ncols=1, left=.505, bottom=0, right=.975,top=.4)
    ax2= fig.add_subplot(gs2[0])
    
    *_,animals2=loc, size, animals=plot_lesion_size_loc(root,ax2,profileLesions2,marker='D',edgecolors='none',linewidths=.1,alpha=.8)
    
    ax2.set_title(f'Late lesion ($n={len(animals2)}$)', fontsize='xx-small')
    
    ax2.set_ylabel('')
    ax2.set_yticklabels([])
    
    
        
    
#     fig.align_ylabels([ax3,ax5])
    ############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax0_,ax0,ax1,ax2)
    OFFX=np.array([.07]*len(AXES))
    OFFY=np.array([.04]*len(AXES))
    OFFY[[2,3]]=0
#     OFFX[[0,1,2,4,6]]=0.05
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LesionSizeLocation.pdf'),
                format='pdf', bbox_inches='tight')
    
    thisPath  =os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LesionSizeLocation.pdf')
    lesionPath=os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Rat276_25.pdf')
    skullPath=os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LesionSites.pdf')
    if os.path.exists(lesionPath) and os.path.exists(skullPath):
        f1=ppdf.PdfFileReader(thisPath).getPage(0)
        f2=ppdf.PdfFileReader(skullPath).getPage(0)
        f3=ppdf.PdfFileReader(lesionPath).getPage(0)

        f1.mergeTranslatedPage(page2=f2, tx=12, ty=140, expand=False)
        f1.mergeTranslatedPage(page2=f3, tx=195, ty=143, expand=False)

        writer=ppdf.PdfFileWriter()
        writer.addPage(f1)
        with open(thisPath,'wb') as f4:
            writer.write(f4)

    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()